In [1]:
import pandas as pd
from dotenv import load_dotenv
import os
from api import get_token, get_camera_token
from pyroclient import Client
import glob
from PIL import Image
import numpy as np
import io
import requests
import random
import shutil
from utils import read_pred_file
import time

In [2]:
cat ../.env

API_URL=http://pyro_api:5050
CAM_USER=my_dummy_login
CAM_PWD=my_dummy_pwd

LOKI_URL=http://my-loki-service.com
PROMTAIL_DEVICE_SCOPE=tower_scope
PROMTAIL_DEVICE_NAME=tower_name

SUPERADMIN_LOGIN=dummy_login
SUPERADMIN_PWD=dummy&P@ssw0rd!

POSTGRES_USER=dummy_pg_user
POSTGRES_PASSWORD=dummy_pg_pwd
POSTGRES_DB=dummy_pg_db

S3_ENDPOINT_URL=http://localstack:4566
S3_ACCESS_KEY=fake
S3_SECRET_KEY=fake
S3_REGION=us-east-1

DB_UI_MAIL=dummy@pyronear.com
DB_UI_PWD=strong-password


In [3]:
API_URL = "http://api:5050"
load_dotenv("../.env")
SUPERADMIN_LOGIN = os.environ.get("SUPERADMIN_LOGIN")
SUPERADMIN_PWD = os.environ.get("SUPERADMIN_PWD")

# Get access token
admin_access_token = get_token(API_URL, SUPERADMIN_LOGIN, SUPERADMIN_PWD)


In [8]:
users = pd.read_csv("../data/csv/API_DATA_DEV - users.csv")
cameras = pd.read_csv("../data/csv/API_DATA_DEV - cameras.csv")
users

,id,organization_id,password,login,role,created_at
0,2,2,test,test77,agent,2024-02-23T08:18:45.447773
1,3,3,test,test07,agent,2024-02-23T08:18:45.447773


Select user via its login

In [9]:
user = "test77"
organization_id = users[users["login"]==user]["organization_id"].item()

user_cameras = cameras[cameras["organization_id"]==organization_id]
user_cameras

,id,organization_id,name,angle_of_view,elevation,lat,lon,is_trustable,created_at
0,2,2,videlles-01,54.2,110,48.4783,2.4242,True,2023-11-07T15:07:19.226673
1,3,2,videlles-02,54.2,110,48.4783,2.4242,True,2023-11-07T15:07:19.226673
2,4,2,croix-augas-01,54.2,110,48.4267,2.7109,True,2023-11-07T15:07:19.226673
3,5,2,croix-augas-02,54.2,110,48.4267,2.7109,True,2023-11-07T15:07:19.226673
4,6,2,moret-sur-loing-01,54.2,110,48.3792,2.8208,True,2023-11-07T15:07:19.226673
5,7,2,moret-sur-loing-02,54.2,110,48.3792,2.8208,True,2023-11-07T15:07:19.226673
6,8,2,nemours-01,54.2,110,48.2605,2.7064,True,2023-11-07T15:07:19.226673
7,9,2,nemours-02,54.2,110,48.2605,2.7064,True,2023-11-07T15:07:19.226673


# Send some alerts
Download a file containing relevant images and predictions, then send alerts to selected cameras (with a random azimuth).

In [10]:
send_alert_from_cam_ids = [2, 4, 5, 7] # select cameras

In [14]:
if not os.path.isdir("selection-true-positives"):
    print("Images not found, dowloading ...")
    url = "https://github.com/pyronear/pyro-envdev/releases/download/v0.0.1/selection-true-positives.zip"
    output_path = "selection-true-positives.zip"

    response = requests.get(url, stream=True)
    response.raise_for_status()  # Raises an error for bad status codes

    with open(output_path, 'wb') as f:
        for chunk in response.iter_content(chunk_size=8192):
            f.write(chunk)

    zip_path = "selection-true-positives.zip"
    extract_dir = "selection-true-positives"  # Current directory

    shutil.unpack_archive(zip_path, extract_dir, 'zip')

    print("Extraction completed.")

    

In [15]:
sequances_folders = glob.glob("selection-true-positives/*")
len(sequances_folders)

12

In [18]:
for camera_id in send_alert_from_cam_ids:
    
    camera_token = get_camera_token(API_URL, camera_id, admin_access_token)
    camera_client  = Client(camera_token, API_URL)

    sequances_folder = sequances_folders[camera_id]

    imgs = glob.glob(f"{sequances_folder}/images/*")
    imgs.sort()
    preds = glob.glob(f"{sequances_folder}/labels_predictions/*")
    preds.sort()

    cam_center_azimuth = random.randint(0,360)
    print(f"Sending alerts from camera {camera_id} at azimuth {cam_center_azimuth}")
    for img_file, pred_file in zip(imgs, preds):
    
        stream = io.BytesIO()
        im = Image.open(img_file)
        im.save(stream, format="JPEG", quality=80)

        bboxes = read_pred_file(pred_file)
        response = camera_client.create_detection(stream.getvalue(), cam_center_azimuth, bboxes)
        # Force a KeyError if the request failed
        
        response.json()["id"]

Sending alerts from camera 2 at azimuth 272
Sending alerts from camera 4 at azimuth 160
Sending alerts from camera 5 at azimuth 210
Sending alerts from camera 7 at azimuth 135


# Create 2 sequences that triangulate
Download a file containning 2 alerts from 2 cameras that triangulate, then it send alerts to the api. 

In [8]:
if not os.path.isdir("triangulated_sequences"):
    print("Images not found, dowloading ...")
    output_path = "triangulated_sequences.zip"

    url = "https://github.com/pyronear/pyro-envdev/releases/download/v0.0.1/"+"triangulated_sequences.zip"

    response = requests.get(url, stream=True)
    response.raise_for_status()  # Raises an error for bad status codes

    with open(output_path, 'wb') as f:
        for chunk in response.iter_content(chunk_size=8192):
            f.write(chunk)

    zip_path = output_path
    extract_dir = "triangulated_sequences"  # Current directory

    shutil.unpack_archive(zip_path, extract_dir, 'zip')

    print("Extraction completed.")

Images not found, dowloading ...
Extraction completed.


In [9]:
cam_triangulation = {
    "12": {
        "azimuth": 226,
        "path": "triangulated_sequences/brison-03"
    },
    "13": {
        "azimuth": 54,
        "path": "triangulated_sequences/serre-de-barre-01"
    }
}

for id, info in cam_triangulation.items():

    camera_id = id
    
    camera_token = get_camera_token(API_URL, camera_id, admin_access_token)
    camera_client  = Client(camera_token, API_URL)

    seq_folder = info["path"]

    imgs = glob.glob(f"{seq_folder}/images/*")
    imgs.sort()
    preds = glob.glob(f"{seq_folder}/labels_predictions/*")
    preds.sort()

    cam_center_azimuth = info["azimuth"]
    print(f"Creating sequences from camera {camera_id} at azimuth {cam_center_azimuth}")

    for img_file, pred_file in zip(imgs, preds):
    
        stream = io.BytesIO()
        im = Image.open(img_file)
        im.save(stream, format="JPEG", quality=80)

        with open(pred_file, 'r') as file:
            bboxes = file.read()

        response = camera_client.create_detection(stream.getvalue(), cam_center_azimuth, eval(bboxes))
        
        time.sleep(1)
        # Force a KeyError if the request failed
        response.json()["id"]

Creating sequences from camera 12 at azimuth 226
Creating sequences from camera 13 at azimuth 54
